In [1]:
!pip install -qq transformers genomic-benchmarks datasets

     |████████████████████████████████| 4.4 MB 15.2 MB/s 
     |████████████████████████████████| 362 kB 58.4 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.2 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 2.3 MB 44.6 MB/s 
     |████████████████████████████████| 271 kB 15.3 MB/s 
     |████████████████████████████████| 140 kB 49.6 MB/s 
     |████████████████████████████████| 1.1 MB 48.2 MB/s 
     |████████████████████████████████| 212 kB 32.2 MB/s 
     |████████████████████████████████| 127 kB 48.4 MB/s 
     |████████████████████████████████| 94 kB 1.0 MB/s 
     |████████████████████████████████| 144 kB 43.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 whic

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
### Parameters
MODEL_NAME = "davidcechak/DNADebertaK8" #Original DNABert model
TOKENIZER_NAME = "armheb/DNA_bert_6"
K = 8
STRIDE = 1

# if less than 1, only this fraction of each dataset is used
DATASET_THINING = 1

BENCHMARKS_FOLDER = '/root/.genomic_benchmarks'
# BENCHMARKS_FOLDER = '/home/jovyan/.genomic_benchmarks/' (for INFRA HUB)

DATASETS = [('demo_coding_vs_intergenomic_seqs', 0),
 ('demo_human_or_worm', 0), ('human_enhancers_cohn', 0), ('human_nontata_promoters', 0)]

BATCH_SIZE = 4
ACCUMULATION = 16

LEARNING_RATE = 1e-5
EPOCHS = 4
RUNS = 5

# do not forget to attach drive
OUTPUT_PATH = 'drive/MyDrive/genomic_benchmarks/RandomizedDNADebertaK8_2.csv'

## Download benchmark datasets and tokenizer

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)

In [3]:
from itertools import product

alphabet = ('A', 'C', 'T', 'G')
vocab = list(map(''.join, product(alphabet, repeat=K)))

tokenizer.add_tokens(vocab)

65536

In [4]:
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check.info import is_downloaded
from pathlib import Path
from tqdm.autonotebook import tqdm

for dataset_name, dataset_version in tqdm(DATASETS):
    if not is_downloaded(dataset_name):
        download_dataset(dataset_name, version=dataset_version, use_cloud_cache=True)

benchmark_root = Path(BENCHMARKS_FOLDER)

  0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
def kmers_strideK(s, k=K):
    return [s[i:i + k] for i in range(0, len(s), k) if i + k <= len(s)]

def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

if (STRIDE == 1):
  kmers = kmers_stride1
else:
  kmers = kmers_strideK

# function used for the actual tokenization
def tok_func(x): return tokenizer(" ".join(kmers(x["seq"])))

# example
tok_func({'seq': 'ATGGAAAGAGGCACCATTCT'})    

{'input_ids': [2, 16136, 52241, 65592, 53460, 4930, 7417, 17366, 57162, 19737, 66647, 57679, 21806, 9387, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

## Looping through datasets, fine-tuning the model for each of them, logging metrics

In [6]:
import pandas as pd
import numpy as np
from random import random, randrange
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

def compute_metrics_binary(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def compute_metrics_multi(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

outputs = []

for dataset_name, dataset_version in tqdm(DATASETS):
    

    labels = sorted([x.stem for x in (benchmark_root / dataset_name / 'train').iterdir()])

    tmp_dict = {}

    for split in ['train', 'test']:
        for nlabel, label in enumerate(labels):
            for f in (benchmark_root / dataset_name / split / label).glob('*.txt'):
                txt = f.read_text()
                if not DATASET_THINING or DATASET_THINING==1:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)
                elif random() < DATASET_THINING:
                    tmp_dict[f"{label} {f.stem}"] = (split, nlabel, txt)

    df = pd.DataFrame.from_dict(tmp_dict).T.rename(columns = {0: "dset", 1: "cat", 2: "seq"})

    ds = Dataset.from_pandas(df)

    tok_ds = ds.map(tok_func, batched=False, remove_columns=['__index_level_0__', 'seq'])
    tok_ds = tok_ds.rename_columns({'cat':'labels'})

    dds = DatasetDict({
        'train': tok_ds.filter(lambda x: x["dset"] == "train").remove_columns('dset'),
        'test':  tok_ds.filter(lambda x: x["dset"] == "test").remove_columns('dset')
    })

    compute_metrics = compute_metrics_binary if len(labels) == 2 else compute_metrics_multi

    for _ in range(RUNS):

        model_cls = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(labels))
        model_cls.deberta.encoder.layer.apply(model_cls.deberta._init_weights)

        args = TrainingArguments('outputs', learning_rate=LEARNING_RATE, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
            evaluation_strategy="epoch", per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE*2, gradient_accumulation_steps=ACCUMULATION,
            num_train_epochs=EPOCHS, weight_decay=0.01, save_steps=100000, seed=randrange(1,10001), report_to='none')
        
        trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                          tokenizer=tokenizer, compute_metrics=compute_metrics)
        trainer.train()
        
        max_accuracy = max([x['eval_accuracy'] for x in trainer.state.log_history if 'eval_accuracy' in x])
        max_f1 = max([x['eval_f1'] for x in trainer.state.log_history if 'eval_f1' in x]) if len(labels) == 2 else np.nan
        train_runtime = max([x['train_runtime'] for x in trainer.state.log_history if 'train_runtime' in x])
        
        outputs.append((dataset_name, max_accuracy, max_f1, train_runtime))
        outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
        outputs_df.to_csv(OUTPUT_PATH, index=False)




  0%|          | 0/4 [00:00<?, ?it/s]

Parameter 'function'=<function tok_func at 0x7f92ae0bb050> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/100000 [00:00<?, ?ex/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

...

***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/davidcechak/DNADebertaK8/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/808901853890ddb7e1ac8c4ecf5112272702596ee51b69cdf389c63da11fe2e9.b628e4d5e7f344695b21fd7ca542aa0f8188268bc6c6b3380dc1e19a16c2447e
Model config DebertaConfig {
  "_name_or_path": "davidcechak/DNADebertaK8",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embedding

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.326793,0.866394,0.869415
1,0.442400,0.190418,0.936019,0.939375
2,0.163600,0.156666,0.953952,0.957263
3,0.068200,0.159728,0.955391,0.958535


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


loading configuration file https://huggingface.co/davidcechak/DNADebertaK8/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/808901853890ddb7e1ac8c4ecf5112272702596ee51b69cdf389c63da11fe2e9.b628e4d5e7f344695b21fd7ca542aa0f8188268bc6c6b3380dc1e19a16c2447e
Model config DebertaConfig {
  "_name_or_path": "davidcechak/DNADebertaK8",
  "architectures": [
    "DebertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embedding

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,No log,0.340525,0.860084,0.859337
1,0.442400,0.181943,0.937791,0.941263
2,0.173900,0.149620,0.953398,0.956620
3,0.071100,0.151546,0.955612,0.958604


***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8
***** Running Evaluation *****
  Num examples = 9034
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




## Outputs

In [7]:
outputs_df = pd.DataFrame(outputs, columns = ['dataset', 'accuracy', 'f1', 'train_runtime'])
outputs_df

,dataset,accuracy,f1,train_runtime
0,demo_coding_vs_intergenomic_seqs,0.912680,0.912690,3085.3888
1,demo_coding_vs_intergenomic_seqs,0.912880,0.914317,3118.3607
2,demo_coding_vs_intergenomic_seqs,0.912080,0.912762,3127.8818
3,demo_coding_vs_intergenomic_seqs,0.912400,0.913240,3124.0516
4,demo_coding_vs_intergenomic_seqs,0.911960,0.911200,3146.3179
5,demo_human_or_worm,0.960080,0.959713,3151.0799
6,demo_human_or_worm,0.960280,0.960259,3125.1223
7,demo_human_or_worm,0.960360,0.960535,3112.1037
8,demo_human_or_worm,0.960800,0.960800,3131.7306
9,demo_human_or_worm,0.960520,0.960600,3170.2935


In [8]:
outputs_df.groupby('dataset').agg({'accuracy' : ['mean', 'sem'], 'f1' : ['mean','sem'], 'train_runtime': ['mean', 'sem']})

accuracy                  f1            \
                                      mean       sem      mean       sem   
dataset                                                                    
demo_coding_vs_intergenomic_seqs  0.912400  0.000174  0.912842  0.000503   
demo_human_or_worm                0.960408  0.000121  0.960382  0.000188   
human_enhancers_cohn              0.736730  0.000972  0.743535  0.002318   
human_nontata_promoters           0.955856  0.000302  0.958818  0.000309   

                                 train_runtime             
                                          mean        sem  
dataset                                                    
demo_coding_vs_intergenomic_seqs    3120.40016   9.927215  
demo_human_or_worm                  3138.06600  10.221769  
human_enhancers_cohn                1112.40848   0.476513  
human_nontata_promoters             1162.10908   2.787958

In [9]:
# saving outputs to csv file
outputs_df.to_csv(OUTPUT_PATH, index=False)